In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from scipy.signal import find_peaks
from scipy.fftpack import ifft2
import pandas as pd
import time
# import websocket
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# import mouse
from rotpy.system import SpinSystem
from rotpy.camera import CameraList
from numpy import unravel_index

In [16]:
def startDriver():
    """Starts the WebDriver (Chrome) and returns the driver object."""
    driver = webdriver.Chrome(executable_path=r"C:\Users\Webdriver\chromedriver.exe")
    return driver

def getToScreener(driver):
    """Navigates the WebDriver to the specified URL and returns the driver object."""
    URL = 'file:///C:/Users/Bruger/Documents/GitHub/B.Sc.-Touch-Free-Interaction/Website/index.html'
    driver.get(URL)
    return driver

def getToDarkness(driver):
    """Navigates the WebDriver to the specified URL and returns the driver object."""
    URL = 'file:///C:/Users/Bruger/Documents/GitHub/B.Sc.-Touch-Free-Interaction/Website/black.html'
    driver.get(URL)
    return driver

def clickNumber(number, driver):
    """Clicks the specified number on a web page using the WebDriver."""
    button = driver.find_element(By.ID, f'key{number}')
    button.click()

def on_message(ws, message):
    """Prints the received message through WebSocket."""
    print(message)

def power_spectrum(fft):
    """Returns the power spectrum of the 2D-Fourier Transform."""
    return np.abs(fft)**2

def FFT(image):
    """Returns the fast Fourier transform after it is centered."""
    fft = np.fft.fft2(image, s=None, axes=(-2, -1), norm=None)
    fft = np.fft.fftshift(fft, axes=None)
    return fft

def inverse_FFT(pwr_spectrum):
    """Computes the inverse Fourier transform of the power spectrum."""
    fft_array = np.fft.ifftshift(pwr_spectrum, axes=None)
    img = ifft2(fft_array)
    img = np.real(img)
    img = (img - np.min(img)) * (255 / (np.max(img) - np.min(img)))
    return np.abs(img.astype(np.uint8)) ** 2

def maskDots(fft_img, X, Y, radius_squared):
    """Masks the dots in the FFT image."""
    r1 = (np.arange(crop_size)[:, None] - Y - crop_size/2) ** 2 + (np.arange(crop_size) - X - crop_size/2) ** 2
    r2 = (np.arange(crop_size)[:, None] + Y - crop_size/2) ** 2 + (np.arange(crop_size) + X - crop_size/2) ** 2
    mask = np.logical_and(r1 > radius_squared, r2 > radius_squared)
    fft_img[mask] = 0
    fft_copy = fft_img.copy()
    return fft_copy

def maskLine(fft_img, X, Y, radius_squared):
    """Masks the line in the FFT image."""
    r = []
    for i in range(10):
        r.append((np.arange(crop_size)[:, None] - Y + (2*Y/10*i) - crop_size/2) ** 2 + (np.arange(crop_size) - X + (2*X/10*i) - crop_size/2) ** 2)
    center = (np.arange(crop_size)[:, None] - crop_size/2) ** 2 + (np.arange(crop_size) - crop_size/2) ** 2
    mask = r[0] > radius_squared
    for i in range(9):
        if i not in [3, 4, 5, 6]:
            mask = np.logical_and(mask, r[i+1] > radius_squared)
    fft_img[mask] = 0
    fft_copy = fft_img.copy()
    return fft_copy

def maskSquare(a, radius, X, Y):
    """Masks the square in the FFT image."""
    X = int(X + crop_size/2)
    Y = int(Y + crop_size/2)
    b = np.array(a.copy(), dtype=np.uint8)
    return b[Y - radius:Y + radius, X - radius:X + radius]

def mask(fft_img, height, width, X, Y, radius_squared):
    """Masks the FFT image."""
    return maskDots(fft_img, height, width, X, Y, radius_squared)

def planeInteraction(peak):
    """Checks if the plane interaction meets a condition."""
    value = abs(fft_dist[int(peak[1]+crop_size/2)][int(peak[0]+crop_size/2)])
    return value > 1800, value

def keyAreaPressed(img):
    """Returns the pressed key based on the image."""
    keys, points = 3, 10
    height, width = len(img) / keys, len(img[0]) / keys
    total = []
    for h in range(keys-1, -1, -1):
        for w in range(keys):
            value = 0
            for i in range(points):
                for j in range(points):
                    value += img[int(h*height + height/4 + height/2*(i+1)/points)][int(w*width + width/4 + width/2*(i+1)/points)]
            total.append(value)
    ID = f'key{total.index(max(total))+1}'
    return ID

def lowToHigh(currentB, previousB):
    """Checks if a condition is met by comparing two values."""
    passed = False
    if currentB and not previousB:
        passed = True
    return passed

k1 = -0.4  # pincushion distortion
k2 = 0.0
k3 = 0.0
p1 = 0.0
p2 = 0.0
dist_coeffs = np.array([k1, k2, k3, p1, p2], dtype=np.float32)

def distortImage(img):
    """Distorts the image."""
    focal_length = 300
    center_x = img.shape[1] / 2
    center_y = img.shape[0] / 2
    camera_matrix = np.array([[focal_length, 0, center_x],
                              [0, focal_length, center_y],
                              [0, 0, 1]], dtype=np.float32)
    img_distorted = cv2.undistort(img, camera_matrix, dist_coeffs)
    return img_distorted

def findPeakGrid(a):
    """Finds the peak value in a grid."""
    size = len(a)
    b = np.asarray(a)
    return b.max()

def xyPeak(a):
    """Returns the X and Y coordinates of the peak in a grid."""
    peak = findPeakGrid(a)
    return [peak[0]+X-radius, peak[1]+Y-radius]

def cameraInDarkness(cap):
    """Sets camera settings for darkness conditions."""
    cap.set(cv2.CAP_PROP_EXPOSURE, -2)
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 104)
    cap.set(cv2.CAP_PROP_SATURATION, 0)
    cap.set(cv2.CAP_PROP_CONTRAST, 255)
    cap.set(cv2.CAP_PROP_SHARPNESS, 165)
    cap.set(cv2.CAP_PROP_FOCUS, 18)
    cap.set(cv2.CAP_PROP_GAIN, 255)
    cap.set(cv2.CAP_PROP_TILT, 0)
    cap.set(cv2.CAP_PROP_PAN, 0)
    cap.set(cv2.CAP_PROP_ZOOM, 390)

def cameraInLight(cap):
    """Sets camera settings for light conditions."""
    cap.set(cv2.CAP_PROP_EXPOSURE, -2)
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 230)
    cap.set(cv2.CAP_PROP_CONTRAST, 255)
    cap.set(cv2.CAP_PROP_SHARPNESS, 130)
    cap.set(cv2.CAP_PROP_FOCUS, 18)
    cap.set(cv2.CAP_PROP_GAIN, 60)
    cap.set(cv2.CAP_PROP_TILT, -1)
    cap.set(cv2.CAP_PROP_PAN, 0)
    cap.set(cv2.CAP_PROP_ZOOM, 500)
    return cap

def getComputerCamera():
    """Returns the computer's camera object."""
    return cv2.VideoCapture(0)

def getWebCam():
    """Returns the webcam object."""
    return cv2.VideoCapture(1 + cv2.CAP_DSHOW)

def changeCameraSetting(cap):
    """Changes the camera settings."""
    cap.set(cv2.CAP_PROP_SETTINGS, 1)
    return cap

def captureImage(camera):
    """Captures an image from the camera."""
    camera.begin_acquisition()
    image_cam = camera.get_next_image(timeout=5)
    image = image_cam.deep_copy_image(image_cam)
    image_data = image_cam.get_image_data()
    image_height = image.get_height()
    image_width = image.get_width()
    image_cam.release()
    camera.end_acquisition()
    numpy_array = np.array(image_data)
    reshaped_array = numpy_array.reshape((image_height, image_width))
    reshaped_array = reshaped_array
    return resizeImg(reshaped_array)

def initCamera():
    """Initializes the camera."""
    system = SpinSystem()
    cameras = CameraList.create_from_system(system, update_cams=True, update_interfaces=True)
    camera = cameras.create_camera_by_index(0)
    camera.init_cam()
    return camera

def resizeImg(img):
    """Resizes the image."""
    smallest_axis = min(len(img), len(img[0]))
    img = img[100:smallest_axis-100, 100:smallest_axis-100]
    dim = (crop_size, crop_size)
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    return resized


In [20]:
print(maskSquare(fft_dist, radius, X, Y))
print(findPeakGrid(maskSquare(fft_dist, radius, Y, X)))
print(findPeakGrid(fft_dist))

[[242 175 215  32  10 223 226 255]
 [188  10 103 223 176  28 128   7]
 [  5  57  11  34  72  78  59  16]
 [ 64 113 135  78 241  28  50 232]
 [249 118  48 149  97  49 176 198]
 [178  54 124 118  41  58 167 161]
 [ 47  27 193  71 195  30  73  98]
 [103 205 113 144  46 188  38  73]]
254
(58379.96681274647+2418.780735733661j)


C:\Users\Bruger\AppData\Local\Temp\ipykernel_7020\3025726011.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  b = np.array(a.copy(), dtype=np.uint8)


In [4]:
# Mask values to be edited
# Y, X, radius_squared = -20, 135, 1000
Y, X, radius_squared, radius = 7, 13, 40, 4
crop_size = 300

In [5]:
# Start Camera
# cap = getWebCam()
# cameraInDarkness(cap)
# changeCameraSetting(cap)

In [6]:
# Open the keyboard using selenium
driver = startDriver()
keyboard = getToDarkness(driver)

previousClicked = False
firstRun = True
firstImageCaptured = False

while True:
    # Capture frame-by-frame
#     ret, frame = cap.read()
#     frame = frame[0:crop_size, 0:crop_size]

#     # Convert the frame to grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    camera = initCamera()
    gray = captureImage(camera)
    
    while firstRun:
        first_gray = gray
        firstRun = False
    
    # Capture background frame when user presses 'f'
    if cv2.waitKey(1) & 0xFF == ord('f'):
        first_gray = gray
        keyboard = getToScreener(driver)
        firstImageCaptured = True

    # Difference in images
    diff_img = cv2.absdiff(gray, first_gray)
    dist_img = distortImage(gray)
#     dist_img = diff_img

    
    fft_dist = FFT(dist_img)
    #only_mask = maskSquare(fft_dist, radius, X, Y)
    #peak = xyPeak(only_mask)
    
    masked_img = maskDots(fft_dist, X, Y, radius_squared)
    final_img = inverse_FFT(masked_img)
    
    #img1 = np.concatenate((gray,power_spectrum(FFT(gray))),axis=0)
    #img2 = np.concatenate((dist_img,power_spectrum(FFT(dist_img))),axis=0)
    #img3 = np.concatenate((final_img,power_spectrum(masked_img)),axis=0)
    #img_total = np.concatenate((img1,img2,img3),axis=1)
    #cv2.imshow('allWindows', img_total)
    
    cv2.imshow("gray",gray)
    cv2.imshow("power_spectrum",power_spectrum(FFT(gray)))
    cv2.imshow("difference",dist_img)
    cv2.imshow("dist_power",power_spectrum(fft_dist))
    cv2.imshow("final_img",final_img)
    cv2.imshow("masked_power",power_spectrum(masked_img))
    
    
    # Tell if plane is interacted with and which key is pressed
    if firstImageCaptured:
        currentClicked = planeInteraction([X,Y])[0]
        #print(planeInteraction([X,Y])[1])
        if currentClicked:
            #print((keyAreaPressed(final_img)), end="\r")
            if lowToHigh(currentClicked, previousClicked):
                keyboard.find_element(By.ID, keyAreaPressed(final_img)).click()
                #print(currentClicked, previousClicked)
                print(keyAreaPressed(final_img))
        previousClicked = currentClicked

    # Exit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object and close all windows
camera.deinit_cam()
camera.release()
cv2.destroyAllWindows()
keyboard.close()

C:\Users\Bruger\AppData\Local\Temp\ipykernel_7020\3162359811.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Webdriver\chromedriver.exe")


key1


SpinnakerAPIException: Spinnaker: Could not start acquisition. Please try reconnecting the device. [-1003]